首先需要导入 fitparse 库用以处理 fit 文件 （通常为运动设备存储的文件类型，例如在手环上记录一次健身活动通常对应一个 fit 文件）
请确保 fitparse 库已在当前环境中正确安装

In [40]:
import fitparse
import numpy as np

下面导入 fit 文件
请注意，将如下文件地址替换为 fit 文件在您计算机中的地址

In [41]:
fitfile = fitparse.FitFile(r'D:\user\15170\2023-09-15-11-53-18.fit')

遍历 fit 文件，获得每秒的心率值 heart_rate_data

In [42]:
# 生成心率数据的空列表
heart_rate_data = []
# 遍历
for record in fitfile.get_messages('record'):

    for record_data in record:
        if record_data.name == 'heart_rate':
            heart_rate = record_data.value
    
    heart_rate_data.append(heart_rate)

假设第一次心跳发生在 fit 文件所记录活动的 0s
那么有：
$$
t_{hb}(i+1) = t_{hb}(i) + \frac{60}{hr} 
$$
其中 $t_{hb}(i+1)$ 为 $i+1$ 次心跳的时间点， $t_{hb}(i)$ 为 $i$ 次心跳的时间点，$hr$ 为此时的心率

In [43]:
# 生成足够长的矩阵存储心跳时间节点，后面舍弃
beatpoint_raw = []
beatpoint_raw.append(0)
i = 0
while i < len(heart_rate_data):
    beatpoint_raw.append(beatpoint_raw[-1] + 60/heart_rate_data[i])
    # 确定时间步的更新条件
    if 1 + i - beatpoint_raw[-1] <= 60/heart_rate_data[i]:
        i += 1


计算 HRV 
也即计算心跳间隔时间的标准差

In [44]:
beatpoint = np.array(beatpoint_raw)
df_beatpoint = 1000*np.diff(beatpoint)
HRV = np.std(df_beatpoint)
print('Your HRV is:', round(HRV, 2), 'ms')

Your HRV is: 46.55 ms
